In [1]:
import cv2
import numpy as np
import os

In [2]:
def most_frequent(List): 
    counter = 0
    num = List[0] 
      
    for i in List: 
        curr_frequency = List.count(i) 
        if(curr_frequency> counter): 
            counter = curr_frequency 
            num = i 
  
    return num 

In [3]:
# if not os.path.exists("data"):
#     os.makedirs("data")
#     os.makedirs("data/train")
#     os.makedirs("data/test")
#     os.makedirs("data/train/bird")
#     os.makedirs("data/train/dog")
#     os.makedirs("data/train/dragon")
#     os.makedirs("data/train/horse")
#     os.makedirs("data/train/tiger")
#     os.makedirs("data/train/fireball")
#     os.makedirs("data/test/bird")
#     os.makedirs("data/test/dog")
#     os.makedirs("data/test/dragon")
#     os.makedirs("data/test/horse")
#     os.makedirs("data/test/tiger")
#     os.makedirs("data/test/fireball")

In [4]:
mode = 'train'
directory = 'data/'+mode+'/'

cap = cv2.VideoCapture(0)

In [ ]:
while True:
    _, frame = cap.read()
    # Simulating mirror image
    frame = cv2.flip(frame, 1)
    
    # Getting count of existing images
    count = {'bird': len(os.listdir(directory+"0/")),
             'dog': len(os.listdir(directory+"1/")),
             'dragon': len(os.listdir(directory+"2/")),
             'horse': len(os.listdir(directory+"3/")),
             'tiger': len(os.listdir(directory+"4/")),
             'fireball': len(os.listdir(directory+"5/"))}
    
    # Printing the count in each set to the screen
    cv2.putText(frame, "MODE : "+mode, (10, 50), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1)
    cv2.putText(frame, "IMAGE COUNT", (10, 100), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1)
    cv2.putText(frame, "BIRD : "+str(count['bird']), (10, 120), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1)
    cv2.putText(frame, "DOG : "+str(count['dog']), (10, 140), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1)
    cv2.putText(frame, "DRAGON : "+str(count['dragon']), (10, 160), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1)
    cv2.putText(frame, "HORSE : "+str(count['horse']), (10, 180), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1)
    cv2.putText(frame, "TIGER : "+str(count['tiger']), (10, 200), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1)
    cv2.putText(frame, "FIREBALL : "+str(count['fireball']), (10, 220), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1)
    
    # Coordinates of the ROI
    x1 = int(0.5*frame.shape[1])
    y1 = 10
    x2 = frame.shape[1]-10
    y2 = int(0.5*frame.shape[1])
    # Drawing the ROI
    # The increment/decrement by 1 is to compensate for the bounding box
    cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255,0,0) ,1)
    # Extracting the ROI
    roi = frame[y1:y2, x1:x2]
    roi = cv2.resize(roi, (64, 64)) 
 
    cv2.imshow("Frame", frame)
    
    #_, mask = cv2.threshold(mask, 200, 255, cv2.THRESH_BINARY)
    #kernel = np.ones((1, 1), np.uint8)
    #img = cv2.dilate(mask, kernel, iterations=1)
    #img = cv2.erode(mask, kernel, iterations=1)
    # do the processing after capturing the image!
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    _, roi = cv2.threshold(roi, 120, 255, cv2.THRESH_BINARY)
    cv2.imshow("ROI", roi)
#     print(count)
#     
    interrupt = cv2.waitKey(10)

#     if interrupt & 0xFF != 255:
#         print(0xFF)
    if interrupt & 0xFF == 27: # esc key
        break
    if interrupt & 0xFF == ord('0'):
        cv2.imwrite(directory+'0/'+str(count['bird'])+'.jpg', roi)
    if interrupt & 0xFF == ord('1'):
        cv2.imwrite(directory+'1/'+str(count['dog'])+'.jpg', roi)
    if interrupt & 0xFF == ord('2'):
        cv2.imwrite(directory+'2/'+str(count['dragon'])+'.jpg', roi)
    if interrupt & 0xFF == ord('3'):
        cv2.imwrite(directory+'3/'+str(count['horse'])+'.jpg', roi)
    if interrupt & 0xFF == ord('4'):
        cv2.imwrite(directory+'4/'+str(count['tiger'])+'.jpg', roi)
    if interrupt & 0xFF == ord('5'):
        cv2.imwrite(directory+'5/'+str(count['fireball'])+'.jpg', roi)
cap.release()
cv2.destroyAllWindows()

In [20]:
print(0xFF)

255


In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [ ]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), input_shape=(64, 64, 1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=6, activation='softmax')) # softmax for more than 2

# Compiling the CNN
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # categorical_crossentropy for more than 2

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('data/train',
                                                 target_size=(64, 64),
                                                 batch_size=1,
                                                 color_mode='grayscale',
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory('data/test',
                                            target_size=(64, 64),
                                            batch_size=1,
                                            color_mode='grayscale',
                                            class_mode='categorical') 
classifier.fit_generator(
        training_set,
        steps_per_epoch=(1173),                   
        epochs=10,
        validation_data=test_set,
        validation_steps=30)# No of images in test set


# Saving the model
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)
classifier.save_weights('model-bw.h5')

In [3]:
import numpy as np
from keras.models import model_from_json
import operator
import cv2
import sys, os
import time

In [4]:
# Loading the model
json_file = open("model-bw.json", "r")
model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(model_json)
# load weights into new model
loaded_model.load_weights("model-bw.h5")
print("Loaded model from disk")

cap = cv2.VideoCapture(0)

# Category dictionary
categories = {0: 'BIRD', 1: 'DOG', 2: 'DRAGON', 3: 'HORSE', 4: 'TIGER', 5: 'FIREBALL'}
counter = 0
recent_predictions = list()

with open('commands.txt', 'w') as newfile:
    newfile.close()

while True:
    _, frame = cap.read()
    # Simulating mirror image
    frame = cv2.flip(frame, 1)
    text_data = ""
    
   
            
    # Got this from collect-data.py
    # Coordinates of the ROI
    x1 = int(0.5*frame.shape[1])
    y1 = 10
    x2 = frame.shape[1]-10
    y2 = int(0.5*frame.shape[1])
    # Drawing the ROI
    # The increment/decrement by 1 is to compensate for the bounding box
    cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255,0,0) ,1)
    # Extracting the ROI
    roi = frame[y1:y2, x1:x2]
    
    # Resizing the ROI so it can be fed to the model for prediction
    roi = cv2.resize(roi, (64, 64)) 
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    _, test_image = cv2.threshold(roi, 120, 255, cv2.THRESH_BINARY)
    cv2.imshow("test", test_image)
    # Batch of 1
    result = loaded_model.predict(test_image.reshape(1, 64, 64, 1))
    prediction = {'BIRD': result[0][0], 
                  'DOG': result[0][1], 
                  'DRAGON': result[0][2],
                  'HORSE': result[0][3],
                  'TIGER': result[0][4],
                  'FIREBALL': result[0][5]}
    # Sorting based on top prediction
    prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True)
   
    # Displaying the predictions
    
#     with sr.Microphone() as source:
#         r.adjust_for_ambient_noise(source)
#         data = r.record(source, duration=.1)
#         print("hello")
#         try:
#             text_data = r.recognize_google(data)
#         except Exception as e:
#             pass
#     if "summon" in text_data:
#         print("summon now!")
        
        

    outputStr = prediction[0][1]
    
    if outputStr == 1.0:
        stringToPassIntoCVWindow = "" + prediction[0][0] + "- confidence: " + str(prediction[0][1])
        
        
        recent_predictions.append(prediction[0][0])
        if recent_predictions[-1] != prediction[0][0]:
            recent_predictions.clear()
        if counter >= 70:
            if len(set(recent_predictions)) >= 1:
                pre = most_frequent(recent_predictions)
                recent_predictions.clear()
                
                with open('commands.txt', 'a+') as outfile:
                    outfile.write(prediction[0][0] + '\n')
                    outfile.close()
                
                counter = 0
                print("SUMMONING A", pre, "WITH A CONFIDENCE OF" ,outputStr)
                
                print("summon something else!")
                time.sleep(2)
                
        time.sleep(.01)
        
        counter = counter + 1
        
        if((counter - 1 )%10 == 0):
            print(int(6 - ((counter-1)/10)), "time units left on cooldown")
            
        cv2.putText(frame, stringToPassIntoCVWindow, (10, 120), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1) 
        cv2.imshow("Frame", frame)
    else:
        cv2.putText(frame, "Naruto Summoner! - use a hand sign", (10,120), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1)    
        cv2.imshow("Frame", frame) 
     
    
    
    interrupt = cv2.waitKey(10)
    if interrupt & 0xFF == 27: # esc key
        break
        
 
cap.release()
cv2.destroyAllWindows()

Loaded model from disk
6 time units left on cooldown
5 time units left on cooldown
4 time units left on cooldown
3 time units left on cooldown
2 time units left on cooldown
1 time units left on cooldown
0 time units left on cooldown
SUMMONING A BIRD WITH A CONFIDENCE OF 1.0
summon something else!
6 time units left on cooldown
5 time units left on cooldown
4 time units left on cooldown
3 time units left on cooldown
2 time units left on cooldown
1 time units left on cooldown
0 time units left on cooldown
SUMMONING A DOG WITH A CONFIDENCE OF 1.0
summon something else!
6 time units left on cooldown
5 time units left on cooldown
4 time units left on cooldown
3 time units left on cooldown
2 time units left on cooldown
1 time units left on cooldown
0 time units left on cooldown
SUMMONING A DOG WITH A CONFIDENCE OF 1.0
summon something else!
6 time units left on cooldown
5 time units left on cooldown
4 time units left on cooldown
3 time units left on cooldown
2 time units left on cooldown
1 ti

KeyboardInterrupt: 